Robot Localization with Python and Particle Filters
===================================================

Import libraries and load map.

In [62]:
import numpy as np
import cv2

map = cv2.imread("map.png", 0)
HEIGHT,WIDTH = map.shape
print(HEIGHT,WIDTH)
print(map)

rx, ry, rtheta = (WIDTH/4, HEIGHT/4, 0)

711 300
[[ 79  80  82 ... 133 148 156]
 [ 78  79  81 ... 138 156 164]
 [ 76  77  79 ... 147 170 180]
 ...
 [181 181 182 ... 174 172 171]
 [178 178 178 ... 180 179 178]
 [177 177 177 ... 183 182 182]]


Map coordinate system

![title](images/coords.png)

CAUTION: The terrain height at X,Y coordinates is map(Y,X).

Read keyboard input.

In [54]:
STEP = 5
TURN = np.radians(25)

def get_input():
    fwd = 0
    turn = 0
    halt = False
    k = cv2.waitKey(0)
    if k == 122: # If key is Z, go forward
        fwd = STEP
    elif k == 100: # If key is D, turn
        turn = TURN
    elif k == 113: # If key is Q, turn the other side
        turn = -TURN
    else: # If another key, wait
        halt = True
    return fwd, turn, halt

Move the robot, with Gausssian noise.

The terrain is not even in the real-world, so there is an uncertainty on how the robot will move. We can simulate that uncertainty with a gaussian, and estimate the confidence using the standard deviation sigma.

![title](images/gaussian.png)

In [11]:
SIGMA_STEP = 0.5 # Noise on the step forward
SIGMA_TURN = np.radians(5) # Noise on the rotation

def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy = np.random.normal(fwd, SIGMA_STEP, 1)
    rx += fwd_noisy * np.cos(rtheta)
    ry += fwd_noisy * np.sin(rtheta)
    print(f'fwd_noisy={fwd_noisy}')
    
    turn_noisy = np.random.normal(turn, SIGMA_TURN, 1)
    rtheta += turn_noisy
    print(f'fwd_turn={np.degrees(turn_noisy)}')
    return rx, ry, rtheta

Initialize particle cloud.

In [12]:
NUM_PARTICLES = 3000

def init():
    particles = np.random.rand(NUM_PARTICLES, 3) # 3 columns : x, y, theta
    particles *= np.array((WIDTH, HEIGHT, np.radians(360)))
    return particles

Move the particles.

In [13]:
def move_particles(particles, fwd, turn):
    particles[:,0] += fwd * np.cos(particles[:,2])
    particles[:,1] += fwd * np.sin(particles[:,2])
    particles[:,2] += turn
    
    particles[:,0] = np.clip(particles[:,0], 0.0, WIDTH-1) # Prevent from falling oustide of the map
    particles[:,1] = np.clip(particles[:,1], 0.0, HEIGHT-1)
    return particles

Get value from robot's sensor.

In [14]:
SIGMA_SENSOR = 2 # Simulate noise of sensors
def sense(x, y, noisy=False):
    x = int(x)
    y = int(y)
    
    if noisy:
        return np.random.normal(map[y,x], SIGMA_SENSOR, 1)
    return map[y, x]

Compute particle weights to prevent them from having the same importance.

In [15]:
def compute_weights(particles, robot_sensor):    
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        elevation = sense(particles[i,0], particles[i, 1], noisy=False)
        errors[i] = abs(robot_sensor - elevation)
    # Now we add the wheights, that goes up when the error goes down
    weights = np.max(errors) - errors
    weights[
        (particles[:,0] == 0) |
        (particles[:,0] == WIDTH-1) |
        (particles[:,1] == 0) |
        (particles[:,1] == HEIGHT-1)
    ] = 0.0 # If on edge then weight is null
    
    # Increase the sensitivity of the weight values to the terrain elevation
    weights = weights ** 3
    return weights

Resample the particles.

In [16]:
def resample(particles, weights):
    probabilities = weights / np.sum(weights) # normalization
    new_index = np.random.choice(NUM_PARTICLES,
                                 size=NUM_PARTICLES,
                                 p=probabilities)
    # Particles with high weights get resampled many times
    particles = particles[new_index,:]
    return particles

Add noise to the particles.

In [17]:
SIGMA_POS = 2
SIGMA_TURN = np.radians(10)

def add_noise(particles):
    noise = np.concatenate((
        np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_TURN, (NUM_PARTICLES, 1)),
    ),
        axis=1
    )
    
    particles += noise
    return particles

Display robot, particles and best guess.

In [18]:
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)
    
    # Display particles
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            cv2.circle(lmap, 
                       (int(particles[i,0]), int(particles[i,1])), 
                       1, 
                       (255,0,0), 
                       1)
        
    # Display robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0,255,0), 10)

    # Display best guess
    if len(particles) > 0:
        px = np.mean(particles[:,0])
        py = np.mean(particles[:,1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0,0,255), 5)

    cv2.imshow('map', lmap)

Main routine.

In [63]:
particles = init()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)        
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)
    
cv2.destroyAllWindows()

fwd_noisy=[4.87359831]
fwd_turn=[-8.09638159]
fwd_noisy=[5.07582008]
fwd_turn=[5.73021071]
fwd_noisy=[5.03199632]
fwd_turn=[0.22524656]
fwd_noisy=[5.11074814]
fwd_turn=[-0.0614548]
fwd_noisy=[5.07682711]
fwd_turn=[9.93166274]
fwd_noisy=[4.97169866]
fwd_turn=[23.25095219]
fwd_noisy=[5.07966664]
fwd_turn=[-0.79587711]
fwd_noisy=[4.81822914]
fwd_turn=[3.4724948]
fwd_noisy=[4.9753323]
fwd_turn=[-4.1317984]
fwd_noisy=[5.23096172]
fwd_turn=[-7.59133054]
fwd_noisy=[4.29787776]
fwd_turn=[7.171492]
fwd_noisy=[5.01026745]
fwd_turn=[-8.58206987]
fwd_noisy=[4.66745985]
fwd_turn=[-1.18662019]
fwd_noisy=[5.22583585]
fwd_turn=[7.54190586]
fwd_noisy=[4.84690489]
fwd_turn=[-15.75208464]
fwd_noisy=[3.95651845]
fwd_turn=[3.75392522]
fwd_noisy=[4.35598033]
fwd_turn=[5.52669572]
fwd_noisy=[4.60197644]
fwd_turn=[-3.51638628]
fwd_noisy=[4.71356417]
fwd_turn=[11.58900399]
fwd_noisy=[5.35182873]
fwd_turn=[-21.57612969]
fwd_noisy=[3.50238794]
fwd_turn=[-15.50459561]
fwd_noisy=[4.71574792]
fwd_turn=[12.07107162]